In [1]:
import sys, time, pickle
import torch

sys.path.insert(0, '../../Utils')
from global_constants import gpu_device

sys.path.insert(0, '../')
from BERT_models import BERT_QNLI_MODEL

from _utils import sample_random_glue_qnli, get_continuation_mapping, \
                    get_continuous_attributions, get_continuous_raw_inputs, \
                    collect_info_for_metric, save_info, download_HTML

In [2]:
qnli_data_raw, targets, idxs = sample_random_glue_qnli()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e6d0a9c9270b6822.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c46c5e78d2bda808.arrow


In [3]:
model = BERT_QNLI_MODEL()
tokenizer = model.tokenizer

In [4]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import Lime
from captum._utils.models.linear_model import SkLearnLasso
from captum.attr import visualization 

In [6]:
lime = Lime(model, interpretable_model=SkLearnLasso(alpha=0.0003))

In [7]:
def generate_record(raw_datum, target): #raw_datum expected to be a tuple/list of 2 sentences
    #tokenizer operations
    tokenized = tokenizer(raw_datum, truncation=True, return_offsets_mapping=True)
    offset_mappings = tokenized['offset_mapping']
    #concatenate the two offset_mappings together because they are fed in together
    conti_map = get_continuation_mapping(offset_mappings[0]) + get_continuation_mapping(offset_mappings[1])
    #change the first input_id of the second sentence to be the last input_id of the 1st sentence (i.e. an [END] token))
    tokenized_input_ids = tokenized['input_ids'][0] + \
                        [tokenized['input_ids'][1][i] if i != 0 else tokenized['input_ids'][0][-1] \
                         for i in range(len(tokenized['input_ids'][1]))]
    input_ids = torch.tensor(tokenized_input_ids).unsqueeze(0).to(gpu_device)
    detokenized = [t.replace('#', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()

     #categorizing results
    pred_class = 'No Entailment' if pred_prob < 0.5 else 'Entailment' 
    true_class = 'No Entailment' if target < 0.5 else 'Entailment' 
    
    #attribution algorithm working
    attribution = lime.attribute(input_emb, n_samples=5000, show_progress=True, perturbations_per_eval=1000)
    attribution[torch.isnan(attribution)] = 0
    word_attributions = attribution.squeeze(0).sum(dim=1)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'No Entailment' if attr_score < 0.5 else 'Entailment'
    convergence_score = None
    
    if torch.all(torch.isnan(attr_score)):
        print(f'nan attri score')
        word_attributions = torch.zeros(word_attributions.size())
    
    
#     #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
    print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
# #     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input_ids=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(qnli_data_raw, targets), start=1):
#     example_1 = "How many times has the South Florida/Miami area hosted the Super Bowl?"
#     example_2 = "The South Florida/Miami area has previously hosted the event 10 times (tied for most with New Orleans), with the most recent one being Super Bowl XLIV in 2010."
    
#     example_1 = "When did the third Digimon series begin?"
#     example_2 = "Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese."
#     datum_raw, target = [example_1, example_2], 1
    print(f'Raw datum: {datum_raw}') #datum expected to be a list of 2 sentences
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record])) 


Raw datum: ['What would a teacher assess the levels of a student on?', 'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.']
GT target: 0


/home/user/anaconda3/envs/metric/lib/python3.9/site-packages/captum/attr/_core/lime.py:1096: UserWarning: Attempting to construct interpretable model with > 10000 features.This can be very slow or lead to OOM issues. Please provide a featuremask which groups input features to reduce the number of interpretablefeatures. 
  warnings.warn(
Lime attribution: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.53s/it]


word attr tensor([ 7.5461e-01, -3.0925e-02, -1.5113e-01, -8.6343e-02, -1.9287e-02,
        -1.8000e-02,  1.2613e-02, -5.5566e-02, -1.3494e-02,  6.3209e-02,
         4.2473e-02,  2.8217e-02,  8.4276e-03, -1.0741e-01, -1.3196e-01,
         1.2674e-01,  7.2729e-02, -1.0328e-01, -7.4075e-04, -3.5120e-02,
        -1.7827e-02,  6.7633e-02,  1.5794e-01, -2.4314e-02,  6.2301e-02,
         1.4776e-02,  1.7038e-01,  1.9442e-01,  6.4709e-03,  5.7674e-02,
        -7.8514e-02, -3.6575e-03, -4.0663e-02, -1.2943e-01,  1.0602e-01,
        -3.6151e-03, -4.4546e-02,  1.0608e-01, -5.9324e-03,  1.0118e-01,
         1.1696e-01,  1.3627e-02,  5.3589e-02,  2.9317e-02,  3.0544e-02,
         3.8907e-02, -8.4504e-02,  2.4649e-01, -8.4385e-04, -1.5330e-01,
         9.1178e-02,  7.3989e-02, -2.6019e-02,  3.7808e-02,  9.1646e-02,
        -1.3450e-01,  5.3882e-02,  4.0919e-02, -6.1915e-02], device='cuda:0')
conti attr [tensor(0.7546, device='cuda:0'), tensor(-0.0309, device='cuda:0'), tensor(-0.1511, device='cuda:0

/home/user/anaconda3/envs/metric/lib/python3.9/site-packages/captum/attr/_core/lime.py:1096: UserWarning: Attempting to construct interpretable model with > 10000 features.This can be very slow or lead to OOM issues. Please provide a featuremask which groups input features to reduce the number of interpretablefeatures. 
  warnings.warn(


<IPython.core.display.HTML object>
Raw datum: ['What company created Doctor Who?', 'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, short stories, audio books, radio plays, interactive video games, game books, webcasts, DVD extras, and even stage performances.']
GT target: 1


...

/home/user/anaconda3/envs/metric/lib/python3.9/site-packages/captum/attr/_core/lime.py:1096: UserWarning: Attempting to construct interpretable model with > 10000 features.This can be very slow or lead to OOM issues. Please provide a featuremask which groups input features to reduce the number of interpretablefeatures. 
  warnings.warn(


<IPython.core.display.HTML object>
Raw datum: ['If the apparant force of two fermions is attractive, what is the spin function?', 'If two identical fermions (e.g. electrons) have a symmetric spin function (e.g. parallel spins) the spatial variables must be antisymmetric (i.e. they exclude each other from their places much as if there was a repulsive force), and vice versa, i.e. for antiparallel spins the position variables must be symmetric (i.e. the apparent force must be attractive).']
GT target: 1


Lime attribution: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.71s/it]


word attr tensor([ 0.0192,  0.0380,  0.1420,  0.2164, -0.0426,  0.0660, -0.0106, -0.0253,
        -0.0054,  0.0260, -0.1115, -0.0783, -0.0862, -0.2678,  0.1400,  0.0348,
        -0.0047, -0.0415,  0.1713,  0.0256, -0.0706, -0.0396,  0.0175, -0.1527,
        -0.0578, -0.1054, -0.0375, -0.0154, -0.0263,  0.1724, -0.0029,  0.0344,
         0.0210, -0.0861,  0.0978, -0.1109,  0.0955, -0.0199,  0.0958,  0.2416,
         0.0587, -0.0125,  0.0996, -0.0084, -0.0535,  0.0290,  0.1997, -0.0746,
         0.0366,  0.1220, -0.0224, -0.0291,  0.0121,  0.1180, -0.0139,  0.0048,
         0.0472,  0.0612, -0.0710, -0.0311, -0.0046, -0.1525, -0.0010, -0.0170,
         0.0088, -0.0716, -0.0496, -0.0396, -0.0381, -0.0146, -0.1205, -0.0649,
         0.0220,  0.0193, -0.0368,  0.0157,  0.0449, -0.0685, -0.0201, -0.0599,
         0.0597, -0.1634, -0.1503, -0.0246, -0.0272,  0.1238, -0.0287, -0.1160,
        -0.1199, -0.0963,  0.0954,  0.0613, -0.0719, -0.1883, -0.1579,  0.1499,
         0.0157, -0.0606,  0.0

<IPython.core.display.HTML object>


In [9]:
file_name_base = 'LIME'

In [10]:
save_info(idxs, qnli_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, 
          fname=f'../MethodOutputs/{file_name_base}_out.pkl')

{'indices': [4013,
  2281,
  2771,
  5109,
  1852,
  2016,
  642,
  3912,
  4427,
  2330,
  4581,
  2611,
  5186,
  2700,
  4920,
  4335,
  1246,
  2811,
  5290,
  5385,
  4774,
  653,
  2816,
  916,
  3297,
  3459,
  1475,
  1287,
  4548,
  1476,
  2328,
  2208,
  2772,
  5301,
  869,
  4968,
  4456,
  4711,
  1929,
  634,
  3774,
  5389,
  497,
  4901,
  4602,
  831,
  1292,
  1087,
  3159,
  3492],
 'raw_data': [['What would a teacher assess the levels of a student on?',
   'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.'],
  ['What company created Doctor Who?',
   'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, sho

In [11]:
download_HTML(cur_file_name=f'{file_name_base}_QNLI_BERT.ipynb',
              out_file_name=f'{file_name_base}_QNLI_BERT.html')